In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/slice_data.csv')

In [3]:
lst1 = []
year = []
month = []
day = []
hour = []
minute = []
lst2 = []
lst3 = []
lst4 = []
for i in df.index:
    s = df['정보발생일시'][i]
    s = str(s)
    if s[-4:] == '0000':
        lst1.append(s)
        year.append(s[:2])
        month.append(s[2:4])
        day.append(s[4:6])
        hour.append(s[6:8])
        minute.append(s[8:10])
        lst2.append(df['차량속도'][i])
        lst3.append(df['GPS_X'][i])
        lst4.append(df['GPS_Y'][i])

temp_df = pd.DataFrame({'정보발생일시': lst1, '년': year, '월': month, '일': day, '시': hour, '분': minute, '차량속도': lst2, 'GPS_X': lst3, 'GPS_Y': lst4})
temp_df.to_csv('../data/extract_minute_all.csv', index=False)

In [4]:
df = pd.read_csv('../data/extract_minute_all.csv')
df

,정보발생일시,년,월,일,시,분,차량속도,GPS_X,GPS_Y
0,22120105080000,22,12,1,5,8,0,129158670,35236021
1,22120105090000,22,12,1,5,9,0,129158670,35236021
2,22120105100000,22,12,1,5,10,0,129158625,35235989
3,22120105110000,22,12,1,5,11,0,129158625,35235989
4,22120105120000,22,12,1,5,12,0,129158625,35235989
...,...,...,...,...,...,...,...,...,...
99143,22123120590000,22,12,31,20,59,15,129148910,35229622
99144,22123121000000,22,12,31,21,0,21,129149201,35229832
99145,22123121010000,22,12,31,21,1,31,129153218,35231953
99146,22123121020000,22,12,31,21,2,0,129153782,35232362


In [5]:
gps_x = np.array(df['GPS_X']) / 1000000
gps_y = np.array(df['GPS_Y']) / 1000000
gps_x = gps_x.tolist()
gps_y = gps_y.tolist()
df['gps_x'] = gps_x
df['gps_y'] = gps_y
del df['GPS_X']
del df['GPS_Y']

In [6]:
df

,정보발생일시,년,월,일,시,분,차량속도,gps_x,gps_y
0,22120105080000,22,12,1,5,8,0,129.158670,35.236021
1,22120105090000,22,12,1,5,9,0,129.158670,35.236021
2,22120105100000,22,12,1,5,10,0,129.158625,35.235989
3,22120105110000,22,12,1,5,11,0,129.158625,35.235989
4,22120105120000,22,12,1,5,12,0,129.158625,35.235989
...,...,...,...,...,...,...,...,...,...
99143,22123120590000,22,12,31,20,59,15,129.148910,35.229622
99144,22123121000000,22,12,31,21,0,21,129.149201,35.229832
99145,22123121010000,22,12,31,21,1,31,129.153218,35.231953
99146,22123121020000,22,12,31,21,2,0,129.153782,35.232362


In [7]:
g = df.groupby(['일', '시'])
g

In [8]:
g.mean()

정보발생일시     년     월          분       차량속도       gps_x      gps_y
일  시                                                                        
1  0   2.212010e+13  22.0  12.0  13.500000  25.071429  129.135317  35.210029
   4   2.212010e+13  22.0  12.0  42.500000   0.705882  129.158177  35.235711
   5   2.212011e+13  22.0  12.0  32.327160  15.598765  129.142761  35.198267
   6   2.212011e+13  22.0  12.0  29.569832  14.318436  129.167253  35.168476
   7   2.212011e+13  22.0  12.0  25.184397  16.170213  129.138009  35.201583
...             ...   ...   ...        ...        ...         ...        ...
31 19  2.212312e+13  22.0  12.0  31.250000  14.773810  129.159159  35.175570
   20  2.212312e+13  22.0  12.0  27.243750  16.243750  129.153054  35.182763
   21  2.212312e+13  22.0  12.0  30.481132  16.377358  129.142637  35.199308
   22  2.212312e+13  22.0  12.0  32.642857  11.392857  129.167604  35.187847
   23  2.212312e+13  22.0  12.0  21.500000  22.386364  129.136730  35.192983

[608 rows x 7 columns]

In [9]:
mean_velocity = df.groupby(['일', '시'], as_index=False).agg({"년": "first", "월": "first", "일": "first", "시": "first", "차량속도": "mean"})
mean_velocity

,년,월,일,시,차량속도
0,22,12,1,0,25.071429
1,22,12,1,4,0.705882
2,22,12,1,5,15.598765
3,22,12,1,6,14.318436
4,22,12,1,7,16.170213
...,...,...,...,...,...
603,22,12,31,19,14.773810
604,22,12,31,20,16.243750
605,22,12,31,21,16.377358
606,22,12,31,22,11.392857


In [10]:
mean_velocity['차량속도'].max()

36.375

In [11]:
mean_velocity['혼잡도'] = pd.cut(mean_velocity.차량속도, bins=[-1, 15, 25, 37], labels=['정체', '서행', '원활'])
mean_velocity

,년,월,일,시,차량속도,혼잡도
0,22,12,1,0,25.071429,원활
1,22,12,1,4,0.705882,정체
2,22,12,1,5,15.598765,서행
3,22,12,1,6,14.318436,정체
4,22,12,1,7,16.170213,서행
...,...,...,...,...,...,...
603,22,12,31,19,14.773810,정체
604,22,12,31,20,16.243750,서행
605,22,12,31,21,16.377358,서행
606,22,12,31,22,11.392857,정체


In [12]:
mean_velocity.to_csv('../data/mean_velocity_all.csv', index=False)